# Creating a Sampled Dataset

**Learning Objectives**

1. Setup up the environment
1. Sample the natality dataset to create train, eval, test sets
1. Preprocess the data in Pandas dataframe


## Introduction 
In this notebook, we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe for a small, repeatable sample.

We will set up the environment, sample the natality dataset to create train, eval, test splits, and preprocess the data in a Pandas dataframe.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/tree/master/courses/machine_learning/deepdive2/end_to_end_ml/solutions/sample_babyweight.ipynb).

## Set up environment variables and load necessary libraries

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [4]:
!pip install --user google-cloud-bigquery==1.25.0 --use-feature=2020-resolver

**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Import necessary libraries.

In [5]:
from google.cloud import bigquery
import pandas as pd

**Lab Task #1:** Set up environment variables so that we can use them throughout the notebook

In [6]:
%%bash
# TODO 1
# TODO -- Your code here.
export PROJECT=$(gcloud config list project --format "value(core.project)")

echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-01-1f811fe4b1bd


In [7]:
PROJECT = "qwiklabs-gcp-01-1f811fe4b1bd"  # Replace with your PROJECT

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets. Let's create a BigQuery client that we'll use throughout the lab.

In [8]:
bq = bigquery.Client(project = PROJECT)

We need to figure out the right way to divide our hash values to get our desired splits. To do that we need to define some values to hash within the module. Feel free to play around with these values to get the perfect combination.

In [14]:
modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

We can make a series of queries to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly. Therefore, to make our code more compact and reusable, let's define a function to return the head of a dataframe produced from our queries up to a certain number of rows.

In [9]:
def display_dataframe_head_from_query(query, count=10):
    """Displays count rows from dataframe head from query.
    
    Args:
        query: str, query to be run on BigQuery, results stored in dataframe.
        count: int, number of results from head of dataframe to display.
    Returns:
        Dataframe head with count number of results.
    """
    df = bq.query(
        query + " LIMIT {limit}".format(
            limit=count)).to_dataframe()

    return df.head(count)

For our first query, we're going to use the original query above to get our label, features, and columns to combine into our hash which we will use to perform our repeatable splitting. There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are. We will need to include all of these extra columns to hash on to get a fairly uniform spread of the data. Feel free to try less or more in the hash and see how it changes your results.

In [10]:
# Get label, features, and columns to hash and split into buckets
hash_cols_fixed_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0.1
    AND mother_age > 15
    AND plurality > 0
    AND gestation_weeks > 20
"""

display_dataframe_head_from_query(hash_cols_fixed_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,year,month,date,state,mother_birth_state
0,6.250105,False,19,2,34,2001,8,2,GA,GA
1,6.867399,True,35,1,39,2001,2,2,MD,MD
2,6.499227,False,42,1,38,2001,3,6,MN,ND
3,8.375361,False,22,1,40,2001,11,6,NY,NY
4,4.124849,False,23,1,36,2001,10,3,TX,TX
5,6.876218,True,29,1,38,2001,5,6,AZ,NM
6,9.625382,True,25,1,39,2001,11,5,AZ,FL
7,6.878423,False,21,1,36,2001,1,2,GA,GA
8,7.874912,True,25,1,39,2001,2,5,PA,PA
9,6.750554,True,22,1,38,2001,4,4,TN,TN


Using `COALESCE` would provide the same result as the nested `CASE WHEN`. This is preferable when all we want is the first non-null instance. To be precise the `CASE WHEN` would become `COALESCE(wday, day, 0) AS date`. You can read more about it [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions).

Next query will combine our hash columns and will leave us just with our label, features, and our hash values.

In [11]:
data_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    ({CTE_hash_cols_fixed})
""".format(CTE_hash_cols_fixed=hash_cols_fixed_query)

display_dataframe_head_from_query(data_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.686620,True,18,1,43,-3844862859777578430
1,9.360828,True,32,1,41,-2152741494508244169
2,8.437091,False,30,1,39,8853848534078421252
3,6.124442,False,24,1,40,-37625298566398460
4,7.125340,False,26,1,41,5892949138069254944
5,6.944561,False,31,1,40,5787518546660136004
6,6.999677,True,23,1,40,8834092610108129167
7,9.374055,True,34,1,40,-5917009640647502733
8,8.373157,True,33,1,40,-2890037128296213246
9,7.187070,False,23,1,40,4334988609252370242


The next query is going to find the counts of each of the unique 657484 `hash_values`. This will be our first step at making actual hash buckets for our split via the `GROUP BY`.

In [12]:
# Get the counts of each of the unique hash of our splitting column
first_bucketing_query = """
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    ({CTE_data})
GROUP BY
    hash_values
""".format(CTE_data=data_query)

display_dataframe_head_from_query(first_bucketing_query)

,hash_values,num_records
0,-4693523204227821573,40
1,-2041930022921169724,1
2,-6190130997840283969,1307
3,-5127176934561882897,39
4,-1888248553071786964,568
5,-5899606308232716406,1544
6,-4694257942846950625,430
7,-2885485386411533513,1746
8,3820521621488099714,1496
9,8730041800929392116,442


The query below performs a second layer of bucketing where now for each of these bucket indices we count the number of records.

In [15]:
# Get the number of records in each of the hash buckets
second_bucketing_query = """
SELECT
    ABS(MOD(hash_values, {modulo_divisor})) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    ({CTE_first_bucketing})
GROUP BY
    ABS(MOD(hash_values, {modulo_divisor}))
""".format(
    CTE_first_bucketing=first_bucketing_query, modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(second_bucketing_query)

,bucket_index,num_records
0,29,450306
1,38,335867
2,8,368468
3,93,214132
4,37,293274
5,63,352935
6,62,424175
7,59,358854
8,24,350200
9,64,281171


The number of records is hard for us to easily understand the split, so we will normalize the count into percentage of the data in each of the hash buckets in the next query.

In [16]:
# Calculate the overall percentages
percentages_query = """
SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
        SUM(num_records)
    FROM
        ({CTE_second_bucketing})) AS percent_records
FROM
    ({CTE_second_bucketing})
""".format(CTE_second_bucketing=second_bucketing_query)

display_dataframe_head_from_query(percentages_query)

,bucket_index,num_records,percent_records
0,88,421104,0.012841
1,4,395531,0.012061
2,39,222585,0.006787
3,57,450220,0.013728
4,35,248710,0.007584
5,93,214132,0.006529
6,68,196450,0.005990
7,59,358854,0.010942
8,17,221055,0.006741
9,86,272691,0.008315


We'll now select the range of buckets to be used in training.

In [17]:
# Choose hash buckets for training and pull in their statistics
train_query = """
SELECT
    *,
    "train" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= 0
    AND bucket_index < {train_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets)

display_dataframe_head_from_query(train_query)

,bucket_index,num_records,percent_records,dataset_name
0,60,352846,0.010759,train
1,46,279722,0.008530,train
2,75,365037,0.011131,train
3,38,335867,0.010242,train
4,15,261613,0.007977,train
5,20,429698,0.013103,train
6,6,544974,0.016618,train
7,59,358854,0.010942,train
8,21,245476,0.007485,train
9,62,424175,0.012934,train


We'll do the same by selecting the range of buckets to be used evaluation.

In [18]:
# Choose hash buckets for validation and pull in their statistics
eval_query = """
SELECT
    *,
    "eval" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {train_buckets}
    AND bucket_index < {cum_eval_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets,
    cum_eval_buckets=train_buckets + eval_buckets)

display_dataframe_head_from_query(eval_query)

,bucket_index,num_records,percent_records,dataset_name
0,84,339018,0.010338,eval
1,87,520605,0.015875,eval
2,85,365755,0.011153,eval
3,83,408523,0.012457,eval
4,88,421104,0.012841,eval
5,89,254705,0.007767,eval
6,82,465179,0.014185,eval
7,81,232043,0.007076,eval
8,86,272691,0.008315,eval
9,80,310399,0.009465,eval


Lastly, we'll select the hash buckets to be used for the test split.

In [19]:
# Choose hash buckets for testing and pull in their statistics
test_query = """
SELECT
    *,
    "test" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {cum_eval_buckets}
    AND bucket_index < {modulo_divisor}
""".format(
    CTE_percentages=percentages_query,
    cum_eval_buckets=train_buckets + eval_buckets,
    modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(test_query)

,bucket_index,num_records,percent_records,dataset_name
0,98,372272,0.011352,test
1,90,284588,0.008678,test
2,99,221816,0.006764,test
3,92,334396,0.010197,test
4,94,428308,0.013060,test
5,97,477724,0.014567,test
6,96,526189,0.016045,test
7,93,214132,0.006529,test
8,91,330906,0.010090,test
9,95,311464,0.009497,test


In the below query, we'll `UNION ALL` all of the datasets together so that all three sets of hash buckets will be within one table. We added `dataset_id` so that we can sort on it in the query after.

In [20]:
# Union the training, validation, and testing dataset statistics
union_query = """
SELECT
    0 AS dataset_id,
    *
FROM
    ({CTE_train})
UNION ALL
SELECT
    1 AS dataset_id,
    *
FROM
    ({CTE_eval})
UNION ALL
SELECT
    2 AS dataset_id,
    *
FROM
    ({CTE_test})
""".format(CTE_train=train_query, CTE_eval=eval_query, CTE_test=test_query)

display_dataframe_head_from_query(union_query)

,dataset_id,bucket_index,num_records,percent_records,dataset_name
0,1,85,365755,0.011153,eval
1,1,84,339018,0.010338,eval
2,1,87,520605,0.015875,eval
3,1,81,232043,0.007076,eval
4,1,80,310399,0.009465,eval
5,1,83,408523,0.012457,eval
6,1,89,254705,0.007767,eval
7,1,86,272691,0.008315,eval
8,1,82,465179,0.014185,eval
9,1,88,421104,0.012841,eval


Lastly, we'll show the final split between train, eval, and test sets. We can see both the number of records and percent of the total data. It is really close to that we were hoping to get.

In [21]:
# Show final splitting and associated statistics
split_query = """
SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
FROM
    ({CTE_union})
GROUP BY
    dataset_id,
    dataset_name
ORDER BY
    dataset_id
""".format(CTE_union=union_query)

display_dataframe_head_from_query(split_query)

,dataset_id,dataset_name,num_records,percent_records
0,0,train,25702765,0.78375
1,1,eval,3590022,0.10947
2,2,test,3501795,0.10678


Now that we know that our splitting values produce a good global splitting on our data, here's a way to get a well-distributed portion of the data in such a way that the train, eval, test sets do not overlap and takes a subsample of our global splits.

**Lab Task #2:** Sample the natality dataset

In [22]:
# TODO 2
# TODO -- Your code here.
# every_n allows us to subsample from each of the hash values
# This helps us get approximately the record counts we want
every_n = 1000

splitting_string = "ABS(MOD(hash_values, {0} * {1}))".format(every_n, modulo_divisor)


def create_data_split_sample_df(query_string, splitting_string, lo, up):
    """Creates a dataframe with a sample of a data split.

    Args:
        query_string: str, query to run to generate splits.
        splitting_string: str, modulo string to split by.
        lo: float, lower bound for bucket filtering for split.
        up: float, upper bound for bucket filtering for split.
    Returns:
        Dataframe containing data split sample.
    """
    query = "SELECT * FROM ({0}) WHERE {1} >= {2} and {1} < {3}".format(
        query_string, splitting_string, int(lo), int(up))

    df = bq.query(query).to_dataframe()

    return df

train_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=0, up=train_percent)

eval_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent, up=train_percent + eval_percent)

test_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent + eval_percent, up=modulo_divisor)


print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 7688 examples in the train dataset.
There are 1035 examples in the validation dataset.
There are 558 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, we'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [23]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.250105,True,20,1,38,7417214837588700033
1,6.715281,False,38,1,43,-6784884401981100070
2,8.247493,False,28,1,40,-6784884401981100070
3,7.588311,True,29,1,37,-6784884401981100070
4,7.125340,True,30,1,39,5052252520303500071


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [24]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hash_values
count,7688.000000,7688.000000,7688.00000,7688.000000,7.688000e+03
mean,7.272325,28.282388,1.03564,38.711238,-3.012053e+17
std,1.288164,6.076118,0.20154,2.451904,5.595723e+18
min,0.626113,16.000000,1.00000,21.000000,-9.210618e+18
25%,6.624891,23.000000,1.00000,38.000000,-6.781866e+18
50%,7.355723,28.000000,1.00000,39.000000,5.057323e+17
75%,8.062305,33.000000,1.00000,40.000000,4.896699e+18
max,11.563246,48.000000,4.00000,47.000000,9.203641e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

**Lab Task #3:** Preprocess the data in Pandas dataframe

In [25]:
   # TODO 3
   # TODO -- Your code here.
def preprocess(df):
    """ Preprocess pandas dataframe for augmented babyweight data.
    
    Args:
        df: Dataframe containing raw babyweight data.
    Returns:
        Pandas dataframe containing preprocessed raw babyweight data as well
            as simulated no ultrasound data masking some of the original data.
    """
    # Clean up raw data
    # Filter out what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]


    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)",
                    "Twins(2)",
                    "Triplets(3)",
                    "Quadruplets(4)",
                    "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound = df.copy(deep=True)

    # Modify is_male
    no_ultrasound["is_male"] = "Unknown"
    
    # Modify plurality
    condition = no_ultrasound["plurality"] != "Single(1)"
    no_ultrasound.loc[condition, "plurality"] = "Multiple(2+)"

    # Concatenate both datasets together and shuffle
    return pd.concat(
        [df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train, eval, test set and see a small sample of the training data after our preprocessing:

In [26]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [27]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.680007,False,24,Single(1),37,1999681345892500047
1,7.438397,Unknown,20,Single(1),39,3620607574719600019
2,6.124442,Unknown,30,Single(1),37,-6784884401981100070
3,7.061406,Unknown,38,Single(1),44,-6784884401981100070
4,6.834330,Unknown,40,Single(1),36,-5549088115530700048


In [28]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
15371,6.106805,Unknown,24,Single(1),36,505732274561700014
15372,7.837433,False,41,Single(1),39,-6784884401981100070
15373,7.583902,Unknown,20,Single(1),41,-6656071312368300079
15374,2.072345,Unknown,30,Multiple(2+),26,-8717259940738900003
15375,6.999677,Unknown,37,Single(1),40,-5368304561280800027


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [29]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hash_values
count,15376.000000,15376.000000,15376.000000,1.537600e+04
mean,7.272325,28.282388,38.711238,-3.012053e+17
std,1.288122,6.075920,2.451824,5.595541e+18
min,0.626113,16.000000,21.000000,-9.210618e+18
25%,6.624891,23.000000,38.000000,-6.781866e+18
50%,7.355723,28.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,40.000000,4.896699e+18
max,11.563246,48.000000,47.000000,9.203641e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [30]:
# Define columns
columns = ["weight_pounds",
           "is_male",
           "mother_age",
           "plurality",
           "gestation_weeks"]

# Write out CSV files
train_df.to_csv(
    path_or_buf="train.csv", columns=columns, header=False, index=False)
eval_df.to_csv(
    path_or_buf="eval.csv", columns=columns, header=False, index=False)
test_df.to_csv(
    path_or_buf="test.csv", columns=columns, header=False, index=False)

In [ ]:
%%bash
wc -l *.csv

In [ ]:
%%bash
head *.csv

In [ ]:
%%bash
tail *.csv

## Lab Summary: 
In this lab, we set up the environment, sampled the natality dataset to create train, eval, test splits, and preprocessed the data in a Pandas dataframe.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License